In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/amiyaguchi/cs224w/wikipedia-retention


# Spark Preprocessing

In [3]:
from src.data.snap_import_user_projection import UnimodalUserProjection
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

input_path = "data/processed/enwiki-meta-compact"
model = UnimodalUserProjection(spark).extract(input_path).transform()

In [4]:
spark.table("bipartite").cache()
spark.table("bipartite").count()

53595946

In [53]:
spark.table("bipartite").printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- article_id: integer (nullable = true)
 |-- edit_date: string (nullable = true)
 |-- word_count: double (nullable = true)
 |-- num_edits: long (nullable = false)



In [87]:
windowed = spark.sql("""
WITH counts as (
    SELECT
        article_id,
        edit_date,
        count(user_id) as n_users
    FROM bipartite
    group by 1, 2
),
labeled as (
    SELECT
        *,
        -- assign each date a row number
        row_number() over (partition by article_id, order by edit_date) as rownum
    FROM counts
)

nearest as (
    SELECT article_id, edit_date, rownum from labeled
    UNION
    SELECT article_id, edit_date, rownum+1 as rownum
    FROM labeled
    WHERE n_users = 1
)
""")